In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from tqdm import tqdm_notebook as tqdm
import requests as req
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 17, 8
rcParams['font.family'] ='AppleGothic'

In [6]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.amazon.com/-/ko/product-reviews/B00CH9QWOU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')


In [7]:
name_list = []
score_list = []
date_list = []
style_list = []
review_list = []

while True:
    try:
        star = bs(driver.page_source, "html.parser")
        score = star.find_all("i",{'data-hook':"review-star-rating"})
        date = driver.find_elements_by_css_selector('span.a-size-base.a-color-secondary.review-date')
        style = driver.find_elements_by_css_selector('a.a-size-mini.a-link-normal.a-color-secondary')
        review = driver.find_elements_by_css_selector("div.a-row.a-spacing-small.review-data")
        time.sleep(1)
        for i in score:
            score_list.append(i.text)
        for i in date:
            date_list.append(i.text)
        for i in style:
            style_list.append(i.text)
        for i in review:
            review_list.append(i.text)
        time.sleep(1)
        btn = driver.find_element_by_css_selector('#cm_cr-pagination_bar > ul > li.a-last')
        btn.click()
        time.sleep(2)
    except:
        break

In [8]:
len(score_list),len(date_list),len(style_list),len(review_list)

(5000, 6002, 4984, 5000)

In [66]:
Coffee1 = {'Star': score_list, 'Review': review_list}

In [67]:
Coffee2 = {'Date': date_list}

In [68]:
Coffee3 = {'Style': style_list}

In [69]:
coffee1 = pd.DataFrame(Coffee1)
coffee2 = pd.DataFrame(Coffee2)
coffee3 = pd.DataFrame(Coffee3)

In [70]:
coffee = pd.concat([coffee1, coffee2, coffee3],axis=1)
coffee = coffee.head(4984)
coffee.to_csv('반자동.csv')

In [71]:
coffee['Star'] = coffee['Star'].str.extract(r'(\d+.\d+)')
coffee

,Star,Review,Date,Style
0,5.0,Despite some of the negative criticism sprinkl...,미국에서 2018년 1월 27일에 검토됨,색상: Brushed Stainless Steel
1,4.0,This is the first espresso machine I've ever o...,미국에서 2017년 6월 16일에 검토됨,색상: Brushed Stainless Steel
2,4.0,Okay so it takes me around 15 minutes but I cl...,미국에서 2018년 1월 27일에 검토됨,색상: Black Sesame
3,5.0,Update 9/19/2020-- The machine continues to ma...,미국에서 2020년 12월 6일에 검토됨,색상: Brushed Stainless Steel
4,5.0,"Update June 2nd 2019: Still use it daily, and ...",미국에서 2022년 8월 14일에 검토됨,색상: Brushed Stainless Steel
...,...,...,...,...
4979,5.0,"If you’re a coffee person, you can’t go wrong ...",미국에서 2022년 7월 2일에 검토됨,색상: Brushed Stainless Steel
4980,5.0,Play Video\n I wanted this for a long time. Wa...,미국에서 2018년 1월 27일에 검토됨,색상: Brushed Stainless Steel
4981,5.0,"Play Video\n I recommend this coffee machine, ...",미국에서 2017년 6월 16일에 검토됨,색상: Brushed Stainless Steel
4982,5.0,Great so far,미국에서 2020년 1월 18일에 검토됨,색상: Brushed Stainless Steel


In [72]:
coffee1 = coffee['Style'].str.split("색상:", expand = True)
coffee1[0] = coffee1[0].str.replace("스타일: ", "")
coffee1.rename(columns={0: "Color", 1: "Style"}, inplace=True)
coffee1.rename(columns={0: "Color"}, inplace=True)
coffee1

,Color,Style
0,,Brushed Stainless Steel
1,,Brushed Stainless Steel
2,,Black Sesame
3,,Brushed Stainless Steel
4,,Brushed Stainless Steel
...,...,...
4979,,Brushed Stainless Steel
4980,,Brushed Stainless Steel
4981,,Brushed Stainless Steel
4982,,Brushed Stainless Steel


In [73]:
ymd = coffee['Date'].str.replace("\w*에서", "")
ymd = ymd.str.replace("Reviewed in the \w* \w* on ", "")
ymd = ymd.str.replace("년 ", "-")
ymd = ymd.str.replace("월 ", "-")
ymd = ymd.str.replace("일", "")
ymd = ymd.str.replace("에 검토됨", "")
ymdf = pd.to_datetime(ymd)
ymdf

0      2018-01-27
1      2017-06-16
2      2018-01-27
3      2020-12-06
4      2022-08-14
          ...    
4979   2022-07-02
4980   2018-01-27
4981   2017-06-16
4982   2020-01-18
4983   2013-11-12
Name: Date, Length: 4984, dtype: datetime64[ns]

In [74]:
coffee1['Star'] = coffee['Star']

In [75]:
coffee1 = coffee1[['Star','Style','Color']]

In [76]:
coffee1['Review'] = coffee['Review']

In [77]:
coffee1['Date'] = ymdf
coffee1

,Star,Style,Color,Review,Date
0,5.0,Brushed Stainless Steel,,Despite some of the negative criticism sprinkl...,2018-01-27
1,4.0,Brushed Stainless Steel,,This is the first espresso machine I've ever o...,2017-06-16
2,4.0,Black Sesame,,Okay so it takes me around 15 minutes but I cl...,2018-01-27
3,5.0,Brushed Stainless Steel,,Update 9/19/2020-- The machine continues to ma...,2020-12-06
4,5.0,Brushed Stainless Steel,,"Update June 2nd 2019: Still use it daily, and ...",2022-08-14
...,...,...,...,...,...
4979,5.0,Brushed Stainless Steel,,"If you’re a coffee person, you can’t go wrong ...",2022-07-02
4980,5.0,Brushed Stainless Steel,,Play Video\n I wanted this for a long time. Wa...,2018-01-27
4981,5.0,Brushed Stainless Steel,,"Play Video\n I recommend this coffee machine, ...",2017-06-16
4982,5.0,Brushed Stainless Steel,,Great so far,2020-01-18


In [78]:
coffee1 = coffee1[['Star','Date','Style','Color','Review']]
coffee1

,Star,Date,Style,Color,Review
0,5.0,2018-01-27,Brushed Stainless Steel,,Despite some of the negative criticism sprinkl...
1,4.0,2017-06-16,Brushed Stainless Steel,,This is the first espresso machine I've ever o...
2,4.0,2018-01-27,Black Sesame,,Okay so it takes me around 15 minutes but I cl...
3,5.0,2020-12-06,Brushed Stainless Steel,,Update 9/19/2020-- The machine continues to ma...
4,5.0,2022-08-14,Brushed Stainless Steel,,"Update June 2nd 2019: Still use it daily, and ..."
...,...,...,...,...,...
4979,5.0,2022-07-02,Brushed Stainless Steel,,"If you’re a coffee person, you can’t go wrong ..."
4980,5.0,2018-01-27,Brushed Stainless Steel,,Play Video\n I wanted this for a long time. Wa...
4981,5.0,2017-06-16,Brushed Stainless Steel,,"Play Video\n I recommend this coffee machine, ..."
4982,5.0,2020-01-18,Brushed Stainless Steel,,Great so far


In [79]:
coffee1 = pd.DataFrame(coffee1)

In [80]:
coffee1 = coffee1.astype({'Star':'float'})
coffee1

,Star,Date,Style,Color,Review
0,5.0,2018-01-27,Brushed Stainless Steel,,Despite some of the negative criticism sprinkl...
1,4.0,2017-06-16,Brushed Stainless Steel,,This is the first espresso machine I've ever o...
2,4.0,2018-01-27,Black Sesame,,Okay so it takes me around 15 minutes but I cl...
3,5.0,2020-12-06,Brushed Stainless Steel,,Update 9/19/2020-- The machine continues to ma...
4,5.0,2022-08-14,Brushed Stainless Steel,,"Update June 2nd 2019: Still use it daily, and ..."
...,...,...,...,...,...
4979,5.0,2022-07-02,Brushed Stainless Steel,,"If you’re a coffee person, you can’t go wrong ..."
4980,5.0,2018-01-27,Brushed Stainless Steel,,Play Video\n I wanted this for a long time. Wa...
4981,5.0,2017-06-16,Brushed Stainless Steel,,"Play Video\n I recommend this coffee machine, ..."
4982,5.0,2020-01-18,Brushed Stainless Steel,,Great so far


In [83]:
coffee = coffee1.drop(['Color'],axis=1)
coffee.to_csv('반자동.csv')

In [84]:
coffee.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4984 entries, 0 to 4983
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Star    4984 non-null   float64       
 1   Date    4984 non-null   datetime64[ns]
 2   Style   4984 non-null   object        
 3   Review  4984 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 194.7+ KB


In [87]:
coffee

,Star,Date,Style,Review
0,5.0,2018-01-27,Brushed Stainless Steel,Despite some of the negative criticism sprinkl...
1,4.0,2017-06-16,Brushed Stainless Steel,This is the first espresso machine I've ever o...
2,4.0,2018-01-27,Black Sesame,Okay so it takes me around 15 minutes but I cl...
3,5.0,2020-12-06,Brushed Stainless Steel,Update 9/19/2020-- The machine continues to ma...
4,5.0,2022-08-14,Brushed Stainless Steel,"Update June 2nd 2019: Still use it daily, and ..."
...,...,...,...,...
4979,5.0,2022-07-02,Brushed Stainless Steel,"If you’re a coffee person, you can’t go wrong ..."
4980,5.0,2018-01-27,Brushed Stainless Steel,Play Video\n I wanted this for a long time. Wa...
4981,5.0,2017-06-16,Brushed Stainless Steel,"Play Video\n I recommend this coffee machine, ..."
4982,5.0,2020-01-18,Brushed Stainless Steel,Great so far
